In [11]:
import cv2
import imutils
import numpy as np
import glob

In [12]:
index = 21
cam = cv2.VideoCapture(0)
while True:
    ret, frame = cam.read()
    frame = imutils.resize(frame, width=1000)
    cv2.imshow("Video", frame)
    key = cv2.waitKey(1) & 0xFF
    if key == ord("c"):
        cv2.imwrite("checkerboard_images/img" + str(index) + ".jpg", frame)
        index = index + 1
    if key == ord("q"):
        cv2.destroyWindow("Video")
        break

In [2]:
# termination criteria
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)

# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,4,0)
objp = np.zeros((5*7,3), np.float32)
objp[:,:2] = np.mgrid[0:7,0:5].T.reshape(-1,2)

# Arrays to store object points and image points from all the images.
objpoints = [] # 3d point in real world space
imgpoints = [] # 2d points in image plane.

images = glob.glob('checkerboard_images/*.jpg')

In [3]:
for fname in images:
    img = cv2.imread(fname)
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

    # Find the chess board corners
    ret, corners = cv2.findChessboardCorners(gray, (7,5),None)

    # If found, add object points, image points (after refining them)
    if ret == True:
        objpoints.append(objp)

        corners2 = cv2.cornerSubPix(gray,corners,(11,11),(-1,-1),criteria)
        imgpoints.append(corners)

        # Draw and display the corners
        cv2.drawChessboardCorners(img, (7,5), corners2,ret)
        #cv2.imshow('img',img)
        #cv2.waitKey(500)

In [7]:
cv2.destroyAllWindows()

In [4]:
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1],None,None)

In [5]:
print("Camera matrix: {}".format(mtx))
print("Distortion coefficients: {}".format(dist))
print("Rotation Vectors: {}".format(rvecs))
print("Translation Vectors: {}".format(tvecs))

Camera matrix: [[ 591.54531435    0.          496.09424034]
 [   0.          590.67079841  303.14255662]
 [   0.            0.            1.        ]]
Distortion coefficients: [[-0.00763408  0.04460509  0.00225208 -0.00070446  0.03251509]]
Rotation Vectors: [array([[-0.21088746],
       [ 0.51610235],
       [-1.41549622]]), array([[-0.21961423],
       [-0.7592079 ],
       [ 1.40679603]]), array([[-0.33901378],
       [ 0.78439131],
       [ 1.30931066]]), array([[ 0.40309526],
       [-0.71017776],
       [-1.36612892]]), array([[ 0.47876923],
       [ 0.19785663],
       [-1.4773774 ]]), array([[ 0.76610045],
       [-0.26437494],
       [-1.40293089]]), array([[ 0.294913 ],
       [ 0.7638158],
       [ 1.0554216]]), array([[ 0.37761146],
       [-0.19341699],
       [ 1.50179793]]), array([[ 0.41747486],
       [-0.45326429],
       [-1.2403339 ]]), array([[-0.12183155],
       [ 0.54549789],
       [ 1.64522841]]), array([[ 0.1333779 ],
       [ 0.18265801],
       [-1.43511663]

In [6]:
img = cv2.imread('checkerboard_images/img8.jpg')
h,  w = img.shape[:2]
newcameramtx, roi=cv2.getOptimalNewCameraMatrix(mtx,dist,(w,h),1,(w,h))

In [10]:
# undistort
dst = cv2.undistort(img, mtx, dist, None, newcameramtx)

# crop the image
x,y,w,h = roi
dst = dst[y:y+h, x:x+w]
cv2.imshow("Result", dst)
cv2.waitKey(0)

13

In [16]:
np.savez('calibration_values', mtx=mtx, dist=dist, rvecs=rvecs, tvecs=tvecs)